Natalie Saechao
HW 6
(1) use runge-kutta method to integrate f(x) = np.exp(-2x), I usde 4th order method

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

#function we want to integrate
def dydx(x,f):
    return np.exp(-2*x)*np.cos(10*x)
#the integral
def f_int(x,C):
    return -(25/24)*((1/10)*np.exp(-2*x)*np.sin(10*x) - (1/50)*np.exp(-2*x)*np.cos(10*x) + C)
#I integrated using wolfram alpha

#defining 4th order method
def rk4_core(x_i, f_i, h, g):
    x_ipoh = x_i + 0.5*h
    x_ipo = x_i + h
    
    k_1 = h*g(x_i,f_i)
    k_2 = h*g(x_ipoh, f_i + 0.5*k_1)
    k_3 = h*g(x_ipoh, f_i + 0.5*k_2)
    k_4 = h*g(x_ipo, f_i + k_3)
    
    f_ipo = f_i + (k_1 + 2*k_2 + 2*k_3 + k_4)/6.
    return f_ipo
def rk4(dydx,a,b,f_a,N):
    x = np.linspace(a,b,N)
    
    h = x[1]-x[0]
    f = np.zeros(N,dtype=float)
    f[0] = f_a #value at lower bound
    
    #evolve f along x
    for i in range(1,N):
        f[i] = rk4_core(x[i-1], f[i-1],h,dydx)
    return x,f
a = 0.0
b = 2*np.pi
N = 10
f_a = 1.0 #plug in x = 0 for f(x), we get 1?
x_4, f_4 = rk4(dydx, a, b, f_a, N)
x = x_4.copy()
plt.plot(x_4, f_4, label = 'RK4')
plt.plot(x,f_int(x,f_a), 'o', label ='analytic')
plt.legend(frameon=False)

(2) Create a notebook to perform runge kutta integration to multiple coupled variables 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

#we have to do several iterative steps along the way because the step size changes

In [ ]:
#defining derivstives to integrate
def dydx(x,y):
    #set the derivatives. our eqn is d^2y/dx^x = -y
    #dydx = z
    #dzdx = -y
    #let y = y(0) and z = y(1)
    
    #declare array of size 2
    y_derivs = np.zeros(2)
   
    y_derivs[0] = y[1]
    y_derivs[1] = -1*y[0]
    
    return y_derivs

In [ ]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    #declare k arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #define x at 0.5 step
    x_ipoh = xi + 0.5*h
    #will advance a step *h
    x_ipo = xi + h
    y_temp = np.zeros(nv) #nv is number of variables
    #get k1 values
    y_derivs = dydx(xi, yi)
    k1[:] = h*y_derivs[:]
    #get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    #get k3 values
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipo, y_temp)
    k4[:] = h*y_derivs[:]
    
    #advance y by a step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    return yipo

In [ ]:
#we want step size, h, to change
def rk4_mv_ad(dydx, x_i,y_i,nv,h,tol):
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    imax = 10000
    i = 0
    #create an error
    Delta = np.full(nv, 2*tol) #limit increase in step, setting error to twice tolerance to start while loop
    #remember the step
    h_step = h
    #awe take one big step h and 2 smaller step h/2
    while(Delta.max()/tol > 1.0):
        #estimate error by taking one step of size h vs two steps of size h/2
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        #compute an error
        Delta = np.fabs(y_2 - y_11)
        if(Delta.max()/tol > 1.0):
            #our error is too large, take a smaller step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i>=imax):
            print("too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i =",i)
            #increase iteration
        i += 1
        #try taking a bigger step, define new step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
        #return answer, a new step, and step we actually took
    return y_2, h_new, h_step


In [ ]:
def rk4_mv(dydx, a, b, y_a, tol):
    #y_a is boundary condition
    
    #define starting step
    xi = a
    yi = y_a.copy()
    #an intial step size == make very small
    h = 1.04e-4 * (b-a)
    #set max # of iterations
    imax = 10000
    i = 0
    #set number of coupled odes to size of y_a
    nv = len(y_a)
    #initial conditions
    x = np.full(1,a) #array with value of a or np.array
    y = np.full((1,nv),y_a) #2d array, a 1 by 2 matrix, initial values of y and z at
    #x = 0 are inside it
    
    #set flags
    flag = 1
    
    #loop until we reach right side
    while(flag):
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        #update the step
        h = h_new
        #prevent an overshoot
        if(xi+h_step>b):
            #take a smaller step
            h = b-xi
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
            #break
            flag = 0
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add steps to the array
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y #: refers to column of array
        y_new[-1,:] = yi[:]
        del y #erase last y matrix, frees memory in y
        y = y_new
        #prevent too many iterations
        if(i>imax):
            print("max iterations reached")
            raise StopIteration("iteration number = ", i)
            
        i += 1
        #output new info
        s = "i = %3d\tx = %9.8f\th =%9.8f\tb=%9.8f" % (i,xi, h_step, b)
        print(s)
            
        #break if new xi is == b
        if(xi==b):
            flag = 0
        #return answer 
    return x, y
    

In [ ]:
import numpy as np
a = 0.0
b = 2.0 * np.pi
y_0 = np.zeros(2) #array of size 2
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2 #number of rows in column
tolerance = 1.0e-6
#error bigger than tolerance because when we define delta

#perform the integration
x,y = rk4_mv(dydx,a,b,y_0,tolerance)


plotting numerical solutions found using runge-kutta, analytical solutions are sin(x) and cos(x).
I am aware that the analytic and numerical solutions do not overlap, and I am not sure why. 
This will result in a large error.



In [ ]:
plt.plot(x,y[:,0],'o', label = 'y')
plt.plot(x,y[:,1],'o', label = 'dy')
xx = np.linspace(0, 2.0*np.pi, 1000)
plt.plot(xx, np.sin(xx), label='analytic y')

plt.plot(xx, np.cos(xx), label='analytic dy')

plt.legend(frameon=False)

In [ ]:
#plotting errors
y_error = (y[:,0]-np.sin(x))
dydx_error = (y[:,1]-np.cos(x))
plt.plot(x, y_error, label="y error")
plt.plot(x, dydx_error, label="dydx error")